In [23]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from sqlalchemy import create_engine

# Census API Key
from config import api_key
c = Census(api_key, year=2017)

## PART 1: Retrieve Census API DATA

In [2]:
# Run Census Search to retrieve data on all zip codes (2017 ACS5 Census)
census_data = c.acs5.get(("NAME", "B01003_001E,B02008_001E,B02009_001E,B06011_001E"), 
                         {'for': 'zip code tabulation area:*'})

In [3]:
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

In [4]:
# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B02008_001E": "White_Pop",
                                      "B02009_001E": "Black_Pop",
                                      "B06011_001E": "Median_Income",
                                      "NAME": "Name", "zip code tabulation area": "ZipCode"})


In [5]:
# Create Column for Percent Non-White (Population-White Pop) / Population
census_pd["Percent_Non_White"] = 100 * \
        ((census_pd["Population"].astype(int) - census_pd["White_Pop"].astype(int)) / \
         census_pd["Population"].astype(int))

In [6]:
# Create Final DataFrame
census_pd = census_pd[["ZipCode", "Population", "White_Pop","Black_Pop","Percent_Non_White",
                       "Median_Income"]]

In [7]:
# Preview Dataframe
print(len(census_pd))
census_pd.head()

33120


ZipCode  Population  White_Pop  Black_Pop  Percent_Non_White  Median_Income
0   00601     17599.0    13798.0      234.0          21.597818            NaN
1   00602     39209.0    36441.0    10348.0           7.059604            NaN
2   00603     50135.0    39939.0     5867.0          20.337090            NaN
3   00606      6304.0     3218.0      234.0          48.953046            NaN
4   00610     27590.0    19671.0     3336.0          28.702428            NaN

In [8]:
# Change ZipCode dtype from objext to integer

census_pd.ZipCode = census_pd.ZipCode.astype(int)
census_pd.dtypes


ZipCode                int64
Population           float64
White_Pop            float64
Black_Pop            float64
Percent_Non_White    float64
Median_Income        float64
dtype: object

## PART 2:  Sort Baltimore CCTV Data

In [9]:
# read in CCTV CSV

cctv_data = "CCTV_Locations.csv"
cctv_df = pd.read_csv(cctv_data)
cctv_df


ID                                        CameraName  \
0    277             733 North Ave West - Etting/Whitelock   
1     71                  70 Downtown - Saratoga/Lexington   
2    376                 1018 Tri-District - Monroe/Ramsay   
3    142           331 Cherry Hill - Rear of 3411 Round Rd   
4    104          221 Broadway - Fleet/Broadway (Eastside)   
5    500                   2013 McCulloh - 1135 Tiffany Ct   
6    357          932 Southwest District - Riggls/Braddish   
7    343      918 Southwest District - Poplar Grove/Riggls   
8    257              713 North Ave West - Pulaski/Herbert   
9    540                     2128 Gilmore - 1649 Lorman Ct   
10   489                    2002 McCulloh - 569 Dolphin St   
11   241                   626 Monument - Ashland/Montford   
12     2               1 Downtown - Eutaw/Lexington Market   
13   579                       2306 Perkins - 231 Mason Ct   
14    91                     208 Broadway - Lombard/Durham   
15   502                   2015 McCulloh - 441 Cummings Ct   
16   363                 1005 Tri-District - Parrish/Ramsy   
17   422                  1303 Harford Rd - Harford/Darley   
18   176                    501 Greenmount - Guilford/28th   
19   546                      2201 Latrobe - 800 Abbott Ct   
20   607                    2413 PVG - NE Elderly Building   
21   119           308 Cherry Hill - Behind 1201 Slater Rd   
22   224                    609 Monument - Monument/Milton   
23   285          801 Park Heights - Reisterstown/Keyworth   
24   317           833 Park Heights - Park Heights/Trainor   
25     7                         6 Downtown - Eutaw/Camden   
26   467           1942 Northeast Market - NE Market North   
27   124                313 Cherry Hill - Spelman/Woodview   
28   255             711 North Ave West - Pulaski/Westwood   
29   391               1102 Pennsylvania Ave - Carey/Baker   
..   ...                                               ...   
584   64                   63 Downtown - Holliday/Saratoga   
585  184                  509 Greenmount - Greenmount/25th   
586  286           802 Park Heights - Reisterstown/Shirley   
587  458              1913 Inner Harbor - Pier 6 Pavillion   
588  523                    2111 Gilmore - 1679 Vincent Ct   
589  380               1022 Tri-District - Pulaski/McHenry   
590  634                    2608 Midtown - Charles/Preston   
591   93                 210 Broadway - Lombard/Washington   
592  115            304 Cherry Hill - Behind 2711 Berea Rd   
593  262             718 North Ave West - Payson/Ridgehill   
594  234                  619 Monument - Mcelderry/Potomac   
595  332  907 Southwest District - Westwood Ave (Dead End)   
596  507                  2020 McCulloh - 1100 Stoddart Ct   
597  347      922 Southwest District - Poplar Grove/Rayner   
598  126           315 Cherry Hill - Rear of Community Ctr   
599  490                    2003 McCulloh - 520 Hoffman St   
600  365                 1007 Tri-District - Gilmor/Ramsay   
601   86                 203 Broadway - Baltimore/Broadway   
602  318             834 Park Heights - Park Heights/Menlo   
603  566                    2221 Latrobe - 1001 McAleer Ct   
604  222                  607 Monument - Monument/Montford   
605  584                     2311 Perkins - 299 Herring Ct   
606  513               2101 Gilmore - Behind 1638 Bruce Ct   
607  229                  614 Monument - Mcelderry/Belnord   
608  618   2502 Dept of Corrections - Madison/Constitution   
609  518                    2106 Gilmore - 1679 Bakbury Ct   
610  473         1962 Avenue Market - Avenue Market Middle   
611  375                1017 Tri-District - Monroe/Wilhelm   
612  239                 624 Monument - Mcelderry/Robinson   
613  103                     220 Broadway - Fleet/Broadway   

                         Owner RegistrationType              ListedDate  \
0         BPD - North Ave West         Business  10/22/2014 05:27:48 PM   
1          

In [10]:
# explore data types

cctv_df.dtypes


ID                               int64
CameraName                      object
Owner                           object
RegistrationType                object
ListedDate                      object
Latitude                       float64
Longitude                      float64
ZipCode                        float64
Neighborhood                    object
PoliceDistrict                  object
CouncilDistrict                  int64
PolicePost                       int64
Location                        object
2010 Census Neighborhoods      float64
2010 Census Wards Precincts    float64
Zip Codes                      float64
dtype: object

In [11]:
# drop any rows where the ZipCode is blank

cctv_df.dropna(subset=['ZipCode'], inplace=True)


In [12]:
# change ZipCode dtype from float to integer (to drop decimal)

cctv_df.ZipCode = cctv_df.ZipCode.astype(int)


In [13]:
# check revised dtypes

cctv_df.dtypes


ID                               int64
CameraName                      object
Owner                           object
RegistrationType                object
ListedDate                      object
Latitude                       float64
Longitude                      float64
ZipCode                          int64
Neighborhood                    object
PoliceDistrict                  object
CouncilDistrict                  int64
PolicePost                       int64
Location                        object
2010 Census Neighborhoods      float64
2010 Census Wards Precincts    float64
Zip Codes                      float64
dtype: object

In [14]:
# preview number of cctv by zipcode

cctv_df['ZipCode'].value_counts()


21201    101
21202     96
21217     83
21225     57
21231     46
21218     44
21223     41
21215     40
21216     37
21205     32
21213     19
21224      5
21230      4
21229      4
21206      1
Name: ZipCode, dtype: int64

In [15]:
# create a new column with number of cameras

cctv_df['NumCameras']= 1


In [16]:
# create a new dataframe groupedby zipcode, dropping all columns except ZipCode & NumCameras

cctv_zips = cctv_df.groupby(['ZipCode'], as_index=False).count()
cctv_zips_raw_df = pd.DataFrame(cctv_zips)
cctv_zips_df = cctv_zips_raw_df.drop(columns=['ID',
                                              'CameraName',
                                              'Owner', 
                                              'RegistrationType',
                                              'ListedDate',
                                              'Latitude',
                                              'Longitude',
                                             'Neighborhood',
                                             'PoliceDistrict',
                                             'CouncilDistrict',
                                             'PolicePost',
                                             'Location',
                                             '2010 Census Neighborhoods',
                                             '2010 Census Wards Precincts',
                                             'Zip Codes'])
cctv_zips_df

ZipCode  NumCameras
0     21201         101
1     21202          96
2     21205          32
3     21206           1
4     21213          19
5     21215          40
6     21216          37
7     21217          83
8     21218          44
9     21223          41
10    21224           5
11    21225          57
12    21229           4
13    21230           4
14    21231          46

In [17]:
# load csv with complete list of Baltimore zip codes

baltimore_zips_data = "all_baltimore_zips.csv"
baltimore_zips_df = pd.read_csv(baltimore_zips_data)
baltimore_zips_df


ZipCode
0     21201
1     21202
2     21203
3     21205
4     21206
5     21207
6     21208
7     21209
8     21210
9     21211
10    21212
11    21213
12    21214
13    21215
14    21216
15    21217
16    21218
17    21222
18    21223
19    21224
20    21225
21    21226
22    21227
23    21229
24    21230
25    21231
26    21233
27    21234
28    21236
29    21237
30    21239
31    21251
32    21263
33    21264
34    21270
35    21273
36    21275
37    21278
38    21279
39    21280
40    21281
41    21287
42    21288
43    21289
44    21290
45    21297
46    21298

In [18]:
# join complete list of zips with list of cctv zips

balt_zips = baltimore_zips_df.join(cctv_zips_df.set_index('ZipCode'), on='ZipCode')


In [19]:
# replace all NaN values with 0 and convert NumCameras column to int

clean_balt_zips = balt_zips.fillna(0)
clean_balt_zips.NumCameras = clean_balt_zips.NumCameras.astype(int)
clean_balt_zips


ZipCode  NumCameras
0     21201         101
1     21202          96
2     21203           0
3     21205          32
4     21206           1
5     21207           0
6     21208           0
7     21209           0
8     21210           0
9     21211           0
10    21212           0
11    21213          19
12    21214           0
13    21215          40
14    21216          37
15    21217          83
16    21218          44
17    21222           0
18    21223          41
19    21224           5
20    21225          57
21    21226           0
22    21227           0
23    21229           4
24    21230           4
25    21231          46
26    21233           0
27    21234           0
28    21236           0
29    21237           0
30    21239           0
31    21251           0
32    21263           0
33    21264           0
34    21270           0
35    21273           0
36    21275           0
37    21278           0
38    21279           0
39    21280           0
40    21281           0
41    21287           0
42    21288           0
43    21289           0
44    21290           0
45    21297           0
46    21298           0

## PART 3: Join the Census & CCTV Dataframes

In [20]:
zips_cctv_data_df = clean_balt_zips.join(census_pd.set_index('ZipCode'), on='ZipCode')
zips_cctv_data_df


ZipCode  NumCameras  Population  White_Pop  Black_Pop  Percent_Non_White  \
0     21201         101     16595.0     6329.0     8749.0          61.862007   
1     21202          96     21788.0     7194.0    13329.0          66.981825   
2     21203           0         NaN        NaN        NaN                NaN   
3     21205          32     14766.0     3636.0    10352.0          75.375863   
4     21206           1     51287.0    12684.0    37531.0          75.268587   
5     21207           0     48243.0     5604.0    41686.0          88.383807   
6     21208           0     36797.0    19907.0    15822.0          45.900481   
7     21209           0     27462.0    20695.0     4542.0          24.641323   
8     21210           0     15258.0    11732.0     1692.0          23.109189   
9     21211           0     16436.0    13355.0     2153.0          18.745437   
10    21212           0     34010.0    18118.0    14416.0          46.727433   
11    21213          19     31733.0     2680.0    28882.0          91.554533   
12    21214           0     20790.0     8842.0    12092.0          57.469937   
13    21215          40     59404.0    10608.0    48625.0          82.142617   
14    21216          37     30037.0     1109.0    29044.0          96.307887   
15    21217          83     36563.0     4093.0    31361.0          88.805623   
16    21218          44     48815.0    15436.0    30964.0          68.378572   
17    21222           0     54771.0    44656.0     7764.0          18.467802   
18    21223          41     23811.0     5028.0    17883.0          78.883709   
19    21224           5     50957.0    36737.0     8918.0          27.905881   
20    21225          57     33001.0    16154.0    14728.0          51.049968   
21    21226           0      7770.0     5677.0     2105.0          26.936937   
22    21227           0     33926.0    26018.0     5735.0          23.309556   
23    21229           4     48225.0     9280.0    37802.0          80.756869   
24    21230           4     34657.0    22913.0     9926.0          33.886372   
25    21231          46     16729.0    10528.0     5273.0          37.067368   
26    21233           0         NaN        NaN        NaN                NaN   
27    21234           0     69649.0    45099.0    20549.0          35.248173   
28    21236           0     39333.0    28130.0     7072.0          28.482445   
29    21237           0     30085.0    17981.0     9337.0          40.232674   
30    21239           0     32065.0     4847.0    26237.0          84.883830   
31    21251           0       881.0      253.0      599.0          71.282633   
32    21263           0         NaN        NaN        NaN                NaN   
33    21264           0         NaN        NaN        NaN                NaN   
34    21270           0         NaN        NaN        NaN                NaN   
35    21273           0         NaN        NaN        NaN                NaN   
36    21275           0         NaN        NaN        NaN                NaN   
37    21278           0         NaN        NaN        NaN                NaN   
38    21279           0         NaN        NaN        NaN                NaN   
39    21280           0         NaN        NaN        NaN                NaN   
40    21281           0         NaN        NaN        NaN                NaN   
41    21287           0         NaN        NaN        NaN                NaN   
42    21288           0         NaN        NaN        NaN                NaN   
43    21289           0         NaN        NaN        NaN                NaN   
44    21290           0         NaN        NaN        NaN                NaN   
45    21297           0         NaN        NaN        NaN                NaN   
46    21298           0         NaN        NaN        NaN                NaN   

    Median_Income  
0         26420.0  
1         22533.0  
2             NaN  
3         17235.0  
4         30133.0  
5         30250.0  
6         40616.0  
7  

In [21]:
# remove any rows missing census data

zips_cctv_data_df.dropna(subset=['Population'], inplace=True)
zips_cctv_data_df


ZipCode  NumCameras  Population  White_Pop  Black_Pop  Percent_Non_White  \
0     21201         101     16595.0     6329.0     8749.0          61.862007   
1     21202          96     21788.0     7194.0    13329.0          66.981825   
3     21205          32     14766.0     3636.0    10352.0          75.375863   
4     21206           1     51287.0    12684.0    37531.0          75.268587   
5     21207           0     48243.0     5604.0    41686.0          88.383807   
6     21208           0     36797.0    19907.0    15822.0          45.900481   
7     21209           0     27462.0    20695.0     4542.0          24.641323   
8     21210           0     15258.0    11732.0     1692.0          23.109189   
9     21211           0     16436.0    13355.0     2153.0          18.745437   
10    21212           0     34010.0    18118.0    14416.0          46.727433   
11    21213          19     31733.0     2680.0    28882.0          91.554533   
12    21214           0     20790.0     8842.0    12092.0          57.469937   
13    21215          40     59404.0    10608.0    48625.0          82.142617   
14    21216          37     30037.0     1109.0    29044.0          96.307887   
15    21217          83     36563.0     4093.0    31361.0          88.805623   
16    21218          44     48815.0    15436.0    30964.0          68.378572   
17    21222           0     54771.0    44656.0     7764.0          18.467802   
18    21223          41     23811.0     5028.0    17883.0          78.883709   
19    21224           5     50957.0    36737.0     8918.0          27.905881   
20    21225          57     33001.0    16154.0    14728.0          51.049968   
21    21226           0      7770.0     5677.0     2105.0          26.936937   
22    21227           0     33926.0    26018.0     5735.0          23.309556   
23    21229           4     48225.0     9280.0    37802.0          80.756869   
24    21230           4     34657.0    22913.0     9926.0          33.886372   
25    21231          46     16729.0    10528.0     5273.0          37.067368   
27    21234           0     69649.0    45099.0    20549.0          35.248173   
28    21236           0     39333.0    28130.0     7072.0          28.482445   
29    21237           0     30085.0    17981.0     9337.0          40.232674   
30    21239           0     32065.0     4847.0    26237.0          84.883830   
31    21251           0       881.0      253.0      599.0          71.282633   

    Median_Income  
0         26420.0  
1         22533.0  
3         17235.0  
4         30133.0  
5         30250.0  
6         40616.0  
7         44300.0  
8         49888.0  
9         38090.0  
10        37016.0  
11        19880.0  
12        34561.0  
13        22306.0  
14        21729.0  
15        18847.0  
16        25490.0  
17        26885.0  
18        15468.0  
19        35246.0  
20        23715.0  
21        32673.0  
22        31299.0  
23        26231.0  
24        42027.0  
25        41724.0  
27        35688.0  
28        38996.0  
29        35113.0  
30        30192.0  
31         3627.0

In [22]:
# change Percent_Non_White dtype from float to integer (to drop decimal)

zips_cctv_data_df.Percent_Non_White = zips_cctv_data_df.Percent_Non_White.astype(int)


In [ ]:
# update columns 

In [63]:
# export to csv

zips_cctv_data_df.to_csv(index=False)
export_zips_cctv_data_csv = zips_cctv_data_df.to_csv (r'/Users/scottgregory/Desktop/Project/export_zips_cctv_data.csv', index = None, header=True)


In [32]:
# create database connection

connection_string = "postgres:chumwater@localhost:5432/baltimore_cctv_db"
engine = create_engine(f'postgresql://{connection_string}')


In [35]:
# Confirm table exists

engine.table_names()


['balt_cctv_census', 'bpd_cctv_census']

In [38]:
# load dataframe into database

zips_cctv_data_df.to_sql(name='bpd_cctv_census', con=engine, if_exists='append', index=False)
zips_cctv_data_df

ZipCode  NumCameras  Population  White_Pop  Black_Pop  Percent_Non_White  \
0     21201         101     16595.0     6329.0     8749.0                 61   
1     21202          96     21788.0     7194.0    13329.0                 66   
3     21205          32     14766.0     3636.0    10352.0                 75   
4     21206           1     51287.0    12684.0    37531.0                 75   
5     21207           0     48243.0     5604.0    41686.0                 88   
6     21208           0     36797.0    19907.0    15822.0                 45   
7     21209           0     27462.0    20695.0     4542.0                 24   
8     21210           0     15258.0    11732.0     1692.0                 23   
9     21211           0     16436.0    13355.0     2153.0                 18   
10    21212           0     34010.0    18118.0    14416.0                 46   
11    21213          19     31733.0     2680.0    28882.0                 91   
12    21214           0     20790.0     8842.0    12092.0                 57   
13    21215          40     59404.0    10608.0    48625.0                 82   
14    21216          37     30037.0     1109.0    29044.0                 96   
15    21217          83     36563.0     4093.0    31361.0                 88   
16    21218          44     48815.0    15436.0    30964.0                 68   
17    21222           0     54771.0    44656.0     7764.0                 18   
18    21223          41     23811.0     5028.0    17883.0                 78   
19    21224           5     50957.0    36737.0     8918.0                 27   
20    21225          57     33001.0    16154.0    14728.0                 51   
21    21226           0      7770.0     5677.0     2105.0                 26   
22    21227           0     33926.0    26018.0     5735.0                 23   
23    21229           4     48225.0     9280.0    37802.0                 80   
24    21230           4     34657.0    22913.0     9926.0                 33   
25    21231          46     16729.0    10528.0     5273.0                 37   
27    21234           0     69649.0    45099.0    20549.0                 35   
28    21236           0     39333.0    28130.0     7072.0                 28   
29    21237           0     30085.0    17981.0     9337.0                 40   
30    21239           0     32065.0     4847.0    26237.0                 84   
31    21251           0       881.0      253.0      599.0                 71   

    Median_Income  
0         26420.0  
1         22533.0  
3         17235.0  
4         30133.0  
5         30250.0  
6         40616.0  
7         44300.0  
8         49888.0  
9         38090.0  
10        37016.0  
11        19880.0  
12        34561.0  
13        22306.0  
14        21729.0  
15        18847.0  
16        25490.0  
17        26885.0  
18        15468.0  
19        35246.0  
20        23715.0  
21        32673.0  
22        31299.0  
23        26231.0  
24        42027.0  
25        41724.0  
27        35688.0  
28        38996.0  
29        35113.0  
30        30192.0  
31         3627.0